In [3]:
import numpy as np
import math
from datascience import *
from scipy import stats

In [4]:
data = Table.read_table('asia-full.csv')
asia = Table()
for label in data.labels:
    clean_col = make_array()
    for i in np.arange(len(data.column(label))):
        if data.column(label).item(i) == '..':
            clean_col = np.append(clean_col, np.nan)
        elif label == 'Country' or label == 'HDI rank':
            clean_col = np.append(clean_col, data.column(label).item(i))   
        else:
            clean_col = np.append(clean_col, float(data.column(label).item(i)))
    asia.append_column(label, clean_col)
asia = asia.take(np.arange(42))
asia

Country,Overall HDI Rank (2015),Within Region HDI Rank (2015),"Maternal mortality ratio (deaths per 100,000 live births)","Adolescent birth rate (births per 1,000 women ages 15–19)",Share of seats in parliament (% held by women),% Female population with at least some secondary education,% Male population with at least some secondary education,Female Labour force participation rate,Male Labour force participation rate
Korea (Republic of),10,1,11,1.62,16.33,88.84,94.56,50.01,71.82
Singapore,11,2,10,3.82,23.91,75.52,81.92,58.24,76.43
Cyprus,21,3,7,4.99,12.5,77.01,82.67,57.47,70.16
Japan,21,3,5,4.07,11.58,93.04,90.64,49.12,70.16
China,37,4,27,7.31,23.62,69.81,79.42,63.58,77.93
Kazakhstan,42,5,12,27.89,20.13,99.68,100,66.07,76.98
United Arab Emirates,46,6,6,29.66,22.5,77.42,64.49,41.9,91.56
Bahrain,48,7,15,13.48,15,61.58,55.56,39.2,85.39
Saudi Arabia,50,8,12,8.82,19.87,63.31,72.13,20.06,79.13
Mongolia,53,9,44,15.67,14.47,89.67,85.83,56.47,68.79


In [5]:
def fhealth(mmr, abr):
    return stats.mstats.gmean([10 / mmr, 1 / abr])

mhealth = 1

In [6]:
def empowerment(pr, se):
    return stats.mstats.gmean([pr, se])

In [7]:
def within_female_across_dimensions(hlth, emp, lbr):    # inputs will be fhealth(..., ...), empowerment(..., ...), flabor
    # gmean of gmeans
    return stats.mstats.gmean([hlth, emp, lbr])

In [8]:
def within_male_across_dimensions(hlth, emp, lbr):
    return stats.mstats.gmean([hlth, emp, lbr])

In [9]:
def across_gender_within_dimension(female, male):
    # harmonic mean
    return stats.mstats.hmean([female, male])

In [12]:
# had to shift row indices by 1 because asia table has 1 less column
def gii_calculator(table):
    def step4():
        return stats.mstats.gmean([np.mean([fhealth(mmr, abr), mhealth]), np.mean([empowerment(fpr, fse), empowerment(mpr, mse)]), np.mean([flabor, mlabor])]) 
    gii_column = make_array()
    for i in np.arange(len(table.column(0))): # row indices of table
        row = table.row(i)
        mmr = row.item(3)
        abr = row.item(4)
        fpr = row.item(5)   # female representation, male = 100 - female
        mpr = 100 - fpr
        # need fse and mse
        fse = row.item(6) 
        mse = row.item(7)
        flabor = row.item(8)
        mlabor = row.item(9)
        gii = 1 - (
        across_gender_within_dimension(
            within_female_across_dimensions(fhealth(mmr, abr), empowerment(fpr, fse), flabor), 
            within_male_across_dimensions(mhealth, empowerment(mpr, mse), mlabor)
        )
        / step4()
        )
        #print(i,gii)
        gii_column = np.append(gii_column, gii)
    with_gii = table.copy().with_column("GII (2016)", gii_column)
    return with_gii

In [13]:
# update table with 2016 GII score
asia = gii_calculator(asia)

In [18]:
asia

Country,Overall HDI Rank (2015),Within Region HDI Rank (2015),"Maternal mortality ratio (deaths per 100,000 live births)","Adolescent birth rate (births per 1,000 women ages 15–19)",Share of seats in parliament (% held by women),% Female population with at least some secondary education,% Male population with at least some secondary education,Female Labour force participation rate,Male Labour force participation rate,GII (2016)
Korea (Republic of),10,1,11,1.62,16.33,88.84,94.56,50.01,71.82,0.0665756
Singapore,11,2,10,3.82,23.91,75.52,81.92,58.24,76.43,0.0682273
Cyprus,21,3,7,4.99,12.5,77.01,82.67,57.47,70.16,0.0990935
Japan,21,3,5,4.07,11.58,93.04,90.64,49.12,70.16,0.0870787
China,37,4,27,7.31,23.62,69.81,79.42,63.58,77.93,0.163752
Kazakhstan,42,5,12,27.89,20.13,99.68,100,66.07,76.98,0.202381
United Arab Emirates,46,6,6,29.66,22.5,77.42,64.49,41.9,91.56,0.195454
Bahrain,48,7,15,13.48,15,61.58,55.56,39.2,85.39,0.233431
Saudi Arabia,50,8,12,8.82,19.87,63.31,72.13,20.06,79.13,0.25692
Mongolia,53,9,44,15.67,14.47,89.67,85.83,56.47,68.79,0.27818


In [22]:
asia.hist("GII (2016)")
#asia.select("GII (2016)").hist()
# why isn't anything plotting

In [28]:
# plot graph
# Circle.map_table(asia) 
# need X,Y latitude,longitude columns for countries in order to use this